# ANN Classification - Bank Customer Retention
## Part 2 - ANN TRAINING (Using Pytorch)
In this notebook, we load the preprocessed training and testing dataset files and train ANN models.

> **INPUT:** the preprocessed training and testing dataset files.<br>
> **OUTPUT:** the trained ANN models.

### 1. INITIALIZATION

In [12]:
# Import necessary libraries and modules
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

In [13]:
# Check pytorch version
torch.__version__

'2.2.2'

### 2. LOADING DATASET FILES

In [14]:
# Prepare file location and load dataset
data_file_location = "..\\data\\interim\\"
data_train_file_name = "churn_modelling_preprocessed_train"
data_test_file_name = "churn_modelling_preprocessed_test"
data_file_ext = "csv"

data_train = pd.read_csv(data_file_location + data_train_file_name + "." + data_file_ext)
data_test = pd.read_csv(data_file_location + data_test_file_name + "." + data_file_ext)

In [15]:
# Check training dataset head
data_train.head()

,Geography_Germany,Geography_Spain,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,-0.569844,1.743090,0.169582,-1.091687,-0.464608,0.006661,-1.215717,0.809503,0.642595,-1.032270,1.106432,0.0
1,1.754865,-0.573694,-2.304559,0.916013,0.301026,-1.377440,-0.006312,-0.921591,0.642595,0.968738,-0.748664,0.0
2,-0.569844,-0.573694,-1.191196,-1.091687,-0.943129,-1.031415,0.579935,-0.921591,0.642595,-1.032270,1.485335,0.0
3,-0.569844,1.743090,0.035566,0.916013,0.109617,0.006661,0.473128,-0.921591,0.642595,-1.032270,1.276528,0.0
4,-0.569844,1.743090,2.056114,-1.091687,1.736588,1.044737,0.810193,0.809503,0.642595,0.968738,0.558378,0.0


In [16]:
# Check testing dataset head
data_test.head()

,Geography_Germany,Geography_Spain,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1.754865,-0.573694,-0.552043,-1.091687,-0.368904,1.044737,0.879303,-0.921591,0.642595,0.968738,1.610857,0.0
1,-0.569844,-0.573694,-1.314903,-1.091687,0.109617,-1.031415,0.429722,-0.921591,0.642595,-1.032270,0.495870,1.0
2,-0.569844,1.743090,0.571630,-1.091687,0.301026,1.044737,0.308583,-0.921591,0.642595,0.968738,-0.424787,0.0
3,-0.569844,-0.573694,1.416961,0.916013,-0.656016,-0.339364,0.575336,-0.921591,-1.556190,-1.032270,-0.187777,0.0
4,1.754865,-0.573694,0.571630,0.916013,-0.081791,0.006661,1.389611,0.809503,0.642595,0.968738,0.616842,0.0


In [17]:
# Split dataset files into independent and dependent features
X_train = data_train.iloc[:,0:-1]
y_train = data_train.iloc[:,-1].values.reshape(-1,1)
X_test = data_test.iloc[:,0:-1]
y_test = data_test.iloc[:,-1].values

In [30]:
# Set data types
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")

### 2. PREPARING DATASET CLASS

In [19]:
# Data set class and its methods
class ChurnModelingData(Dataset):
    
    def __init__(self, X_train, y_train) -> None:
        super().__init__()
        self.X = X_train
        self.y = y_train
    
    def __getitem__(self, index):
        X = torch.tensor(self.X[index], dtype=torch.float32)
        y = torch.tensor(self.y[index], dtype=torch.float32)
        return X, y
    
    def __len__(self):
        return len(self.y)

In [20]:
# Data object and loader
churn_data = ChurnModelingData(X_train.values, y_train)
train_loader = DataLoader(churn_data, batch_size = 32, shuffle = True)

### 3. BUILDING ANN MODEL

In [21]:
# Define ANN class
class CustomerRetentionNetwork(nn.Module):
    
    def __init__(self, NUM_FEATURES):
        super().__init__()
        
        # Add first input layer
        self.lin1 = nn.Linear(NUM_FEATURES, 6)
        
        # Add second hidden layer
        self.lin2 = nn.Linear(6, 6)
        
        # Add output layer
        self.lin3 = nn.Linear(6, 1)
        
        # Add sigmoid function
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = torch.relu(self.lin1(x))
        x = torch.relu(self.lin2(x))
        x = self.lin3(x)
        x = self.sigmoid(x)
        return x

In [22]:
# Initialize hyperparameters
NUM_FEATURES = churn_data.X.shape[1]

In [23]:
# Initialize ann model
ann = CustomerRetentionNetwork(NUM_FEATURES = NUM_FEATURES)

In [24]:
# Loss function: Binary Cross Entropy Loss for binary classification
criterion = nn.BCELoss()

In [25]:
# Optimizer
optimizer = torch.optim.Adam(ann.parameters(), lr = 0.001)

### 4. TRAINING ANN MODEL

In [27]:
# Epochs number
NUM_EPOCHS = 100

# Loss tracking
losses = []

# Training loops
for epoch in range(NUM_EPOCHS):
    ann.train()
    epoch_loss = 0
    
    for X_batch, y_batch in train_loader:
        
        # Initialize gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = ann(X_batch)
        
        # Calculate losses
        loss = criterion(output, y_batch)
        
        # Backward pass
        loss.backward()
        
        # Update weights
        optimizer.step()
        
    losses.append(float(loss.data.detach().numpy()))

### 5. EVALUATION

In [31]:
X_test_torch = torch.from_numpy(X_test.values)
with torch.no_grad():
    y_test_log = ann(X_test_torch)
    y_test_pred = torch.max(y_test_log.data, 1)

In [32]:
accuracy_score(y_test, y_test_pred.indices)

0.7975